In [ ]:
pip install keras

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
pip install tensorflow

Python interpreter will be restarted.
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-9d7c8f72-b051-4248-b859-117ca745b1ce
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.5
    Not uninstalling protobuf at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-9d7c8f72-b051-4248-b859-117ca745b1ce
    Can't uninstall 'protobuf'. No files were found to uninstall.
Python interpreter will be restarted.


In [ ]:
pip install keras-applications

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Create Spark Dataframe") \
    .getOrCreate()

In [ ]:
spark.conf.set("fs.azure.account.key.serengeti.blob.core.windows.net", "")

Database = "dedicateddef"
Server = "serengetidatalab.sql.azuresynapse.net"
User = "user@serengetidatalab"
# Pass = sqladmin00!
JdbcPort =  "1433"
Pass = ""
JdbcExtraOptions = "encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

sqlUrl = f"jdbc:sqlserver://{Server}:{JdbcPort};database={Database};user={User};password={Pass};${JdbcExtraOptions}"

tableName = "train"

tempDir = "abfss://snapshot-serengeti@serengeti.dfs.core.windows.net/"

df_train = spark.read \
  .format("jdbc") \
  .option("url", sqlUrl) \
  .option("tempDir", tempDir) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("dbTable", tableName) \
  .load()

df_train.show()


+--------------------+---------------+
|            image_id|           name|
+--------------------+---------------+
|S4/B06/B06_R2/S4_...|         impala|
|S3/S07/S07_R12/S3...|     wildebeest|
|S3/S07/S07_R12/S3...|     wildebeest|
|S3/U10/U10_R11/S3...|          zebra|
|S3/U10/U10_R11/S3...|     hartebeest|
|SER_S11/P13/P13_R...|        cheetah|
|SER_S11/J09/J09_R...|          zebra|
|S2/K03/K03_R2/S2_...|     wildebeest|
|S6/S13/S13_R2/S6_...|   hyenaspotted|
|S9/I02/I02_R2/S9_...|          zebra|
|S1/O05/O05_R1/S1_...|     guineafowl|
|S10/F06/F06_R1/S1...|     wildebeest|
|S1/Q11/Q11_R1/S1_...|gazellethomsons|
|S1/Q11/Q11_R1/S1_...|gazellethomsons|
|S8/H05/H05_R3/S8_...|     wildebeest|
|S1/R10/R10_R1/S1_...|  gazellegrants|
|S9/K06/K06_R2/S9_...|gazellethomsons|
|S3/E04/E04_R12/S3...|     wildebeest|
|S4/E04/E04_R2/S4_...|         impala|
|S4/E13/E13_R2/S4_...|     wildebeest|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
tableName2 = "val"

df_val = spark.read \
  .format("jdbc") \
  .option("url", sqlUrl) \
  .option("tempDir", tempDir) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("dbTable", tableName2) \
  .load()

df_val.show()

+--------------------+---------------+
|            image_id|           name|
+--------------------+---------------+
|S8/K09/K09_R2/S8_...|          zebra|
|S4/K11/K11_R2/S4_...|     wildebeest|
|S7/L05/L05_R2/S7_...|gazellethomsons|
|S4/M13/M13_R2/S4_...|gazellethomsons|
|S7/M05/M05_R3/S7_...|gazellethomsons|
|S7/M07/M07_R2/S7_...|      otherbird|
|SER_S11/H04/H04_R...|gazellethomsons|
|S5/I12/I12_R1/S5_...|       elephant|
|S2/D03/D03_R3/S2_...|     wildebeest|
|S2/D03/D03_R3/S2_...|          zebra|
|S5/U09/U09_R3/S5_...|   hyenaspotted|
|S3/P05/P05_R11/S3...|gazellethomsons|
|S2/E01/E01_R4/S2_...|          zebra|
|S2/E01/E01_R4/S2_...|          zebra|
|S9/I05/I05_R2/S9_...|     wildebeest|
|SER_S11/Q06/Q06_R...|          zebra|
|S9/J04/J04_R2/S9_...|     wildebeest|
|S9/J04/J04_R2/S9_...|          zebra|
|S9/J04/J04_R2/S9_...|     wildebeest|
|S2/H04/H04_R2/S2_...|          zebra|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
import numpy as np
import keras
from keras.applications import ResNet50
from keras.preprocessing import image
#from keras.applications.resnet50 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
import pandas as pd 



In [ ]:
df_train = df_train.toPandas()
df_val = df_val.toPandas()

In [ ]:
def imageURL(img_id):
    return f"https://lilablobssc.blob.core.windows.net/snapshotserengeti-unzipped/{img_id}.JPG"

df_train["image_link"] = df_train["image_id"].apply(imageURL)
df_val["image_link"] = df_val["image_id"].apply(imageURL)

In [ ]:
import multiprocessing as mp

def load_image(url, queue):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((224, 224), Image.ANTIALIAS)
        image = np.array(img)
        queue.put(image)
    except:
        queue.put(None)
        
def load_images(image_urls):
    manager = mp.Manager()
    queue = manager.Queue()
    processes = []
    
    for url in image_urls:
        process = mp.Process(target=load_image, args=(url, queue))
        process.start()
        processes.append(process)
        
    images = []
    success_count = 0
    failure_count = 0
    
    for _ in range(len(image_urls)):
        image = queue.get()
        if image is not None:
            images.append(image)
            success_count += 1
        else:
            failure_count += 1
            
    for process in processes:
        process.join()
        
    train_data = np.stack(images)
    
    print("Successful loads:", success_count)
    print("Failed loads:", failure_count)
    
    return train_data


In [ ]:
sub_set = df_train.iloc[:4]["image_link"]
sub_set

Out[19]: 0    https://lilablobssc.blob.core.windows.net/snap...
1    https://lilablobssc.blob.core.windows.net/snap...
2    https://lilablobssc.blob.core.windows.net/snap...
3    https://lilablobssc.blob.core.windows.net/snap...
Name: image_link, dtype: object

In [ ]:

train_data = load_images(sub_set)
# val_data = load_images(df_val.iloc[0]["image_link"])

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<command-3812778485697081> in <cell line: 1>()
----> 1 train_data = load_images(sub_set)
      2 # val_data = load_images(df_val.iloc[0]["image_link"])

<command-3812778485697078> in load_images(image_urls)
     36         process.join()
     37 
---> 38     train_data = np.stack(images)
     39 
     40     print("Successful loads:", success_count)

<__array_function__ internals> in stack(*args, **kwargs)

/databricks/python/lib/python3.9/site-packages/numpy/core/shape_base.py in stack(arrays, axis, out)
    421     arrays = [asanyarray(arr) for arr in arrays]
    422     if not arrays:
--> 423         raise ValueError('need at least one array to stack')
    424 
    425     shapes = {arr.shape for arr in arrays}

ValueError: need at least one array to stack

In [ ]:
labels_map = {
    'empty': 0,
    'human': 1,
    'gazellegrants': 2,
    'reedbuck': 3,
    'dikdik': 4,
    'zebra': 5,
    'porcupine': 6,
    'gazellethomsons': 7,
    'hyenaspotted': 8,
    'warthog': 9,
    'impala': 10,
    'elephant': 11,
    'aardvark': 12,
    'giraffe': 13,
    'mongoose': 14,
    'buffalo': 15,
    'hartebeest': 16,
    'guineafowl': 17,
    'wildebeest': 18,
    'leopard': 19,
    'ostrich': 20,
    'lionfemale': 21,
    'koribustard': 22,
    'otherbird': 23,
    'cheetah': 24,
    'honeybadger': 25,
    'bushbuck': 26,
    'jackal': 27,
    'aardwolf': 28,
    'hippopotamus': 29,
    'hyenastriped': 30,
    'hare': 31,
    'baboon': 32,
    'monkeyvervet': 33,
    'batearedfox': 34,
    'waterbuck': 35,
    'secretarybird': 36,
    'topi': 37,
    'serval': 38,
    'lionmale': 39,
    'eland': 40,
    'rodents': 41,
    'wildcat': 42,
    'civet': 43,
    'genet': 44,
    'zorilla': 45,
    'caracal': 46,
    'rhinoceros': 47,
    'reptiles': 48,
    'insectspider': 49,
    'duiker': 50,
    'cattle': 51,
    'vulture': 52,
    'steenbok': 53,
    'bat': 54,
    'fire': 55,
    'hyenabrown': 56,
    'wilddog': 57,
    'kudu': 58,
    'pangolin': 59,
    'lioncub': 60
}

image_labels = ['empty', 'human', 'gazellegrants', 'reedbuck', 'dikdik', 'zebra', 'porcupine', 'gazellethomsons', 'hyenaspotted', 'warthog', 'impala', 'elephant', 'aardvark', 'giraffe', 'mongoose', 'buffalo', 'hartebeest', 'guineafowl', 'wildebeest', 'leopard', 'ostrich', 'lionfemale', 'koribustard', 'otherbird', 'cheetah', 'honeybadger', 'bushbuck', 'jackal', 'aardwolf', 'hippopotamus', 'hyenastriped', 'hare', 'baboon', 'monkeyvervet', 'batearedfox', 'waterbuck', 'secretarybird', 'topi', 'serval', 'lionmale', 'eland', 'rodents', 'wildcat', 'civet', 'genet', 'zorilla', 'caracal', 'rhinoceros', 'reptiles', 'insectspider', 'duiker', 'cattle', 'vulture', 'steenbok', 'bat', 'fire', 'hyenabrown', 'wilddog', 'kudu', 'pangolin', 'lioncub']



In [ ]:
import numpy as np

def one_hot_encode_labels(image_labels, labels_map):
    # Convert the label names to integers using the labels_map dictionary
    label_ints = [labels_map[label] for label in image_labels]
    # Perform one-hot encoding using np.eye
    train_labels = np.eye(len(labels_map))[label_ints]
    return train_labels

train_labels = one_hot_encode_labels(image_labels, labels_map)
